# 垃圾分拣玩法  Garbage sorting play

### 导入头文件 Import head file

In [ ]:
#!/usr/bin/env python
# coding: utf-8
import Arm_Lib
import cv2 as cv
import threading
from time import sleep
import ipywidgets as widgets
from IPython.display import display
from single_garbage_identify import single_garbage_identify

### 创建实例,初始化参数 Create the instance and initialize the parameters

In [ ]:
single_garbage = single_garbage_identify()
model = "General"

### 初始化机械臂位置 Initialize DOFBOT position

In [ ]:
import Arm_Lib
arm = Arm_Lib.Arm_Device()
joints_0 = [90, 90, 15, 20, 90, 30]
arm.Arm_serial_servo_write6_array(joints_0, 1000)

### 创建控件 Creating widget

In [ ]:
button_layout      = widgets.Layout(width='320px', height='60px', align_self='center')
output = widgets.Output()
# 退出
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))

### 抓取控制  grab control

In [ ]:
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output: print(model)
exit_button.on_click(exit_button_Callback)

### 主程序 Main process

In [ ]:
def camera():
    # 打开摄像头 Open camera
    capture = cv.VideoCapture(0)
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            _, img = capture.read()
            img = cv.resize(img, (640, 480))
            img = single_garbage.single_garbage_run(img)
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:capture.release()

### 启动  Start

In [ ]:
# Please place the building block in the center of the cross (the picture is facing the mechanical arm)
# 请将积木块正放在十字中心(图片正对机械臂)
display(controls_box,output)
threading.Thread(target=camera, ).start()